In [ ]:
# core
import numpy as np
import pandas as pd

# utility
from datetime import datetime

# MSE imports
from astro_utils import datetime_to_mjd
from horizons_files import load_pos_jpl, load_ast_pos_jpl, load_obs_jpl, load_ast_obs_jpl
from asteroid_integrate import load_ast_elt
from asteroid_dataframe import load_ast_data, spline_ast_vec, calc_ast_dir, spline_ast_vec_dir
from asteroid_dataframe import calc_ast_data, spline_ast_vec_df
from asteroid_dataframe import compare_df_vec, compare_df_dir
from asteroid_data import orbital_element_batch
from ztf_data import load_ztf_det_all

### Position of Earth according to JPL

In [ ]:
# Data directories
dir_name_hourly = '../data/jpl/testing/hourly'
dir_name_daily = '../data/jpl/testing/daily'

# Build DataFrame for earth and mars position at 3 hour frequency
# df_earth = load_pos_jpl(body_name='earth', dir_name=dir_name_hourly)

# Earth at daily frequency
df_earth_jpl = load_pos_jpl(body_name='earth', dir_name=dir_name_daily)

In [ ]:
df_earth_jpl

### Positions of First 16 Asteroids from JPL

In [ ]:
# Load the asteroid position and velocity from JPL
df_ast_jpl = load_ast_pos_jpl(ast_num0=1, ast_num1=16, dir_name=dir_name_daily)

In [ ]:
df_ast_jpl

### Observations of First 16 Asteroids from JPL

In [ ]:
# Load the asteroid observations from JPL
df_obs_jpl = load_ast_obs_jpl(ast_num0=1, ast_num1=16, observer_name='geocenter', dir_name=dir_name_daily)

In [ ]:
df_obs_jpl

### Position of Asteroids & Earth from MSE Integration

In [ ]:
# alias inputs
n0 = 1
n1 = 17
mjd0 = datetime_to_mjd(datetime(2010,1,1))
mjd1 = datetime_to_mjd(datetime(2020,1,2))

In [ ]:
# Load first block of asteroid data
df_ast_mse, df_earth_mse, df_sun_mse = load_ast_data(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1)

In [ ]:
# DataFrame of asteroid snapshots
ast_elt = load_ast_elt()
# ast_elt

### Check Position of Earth vs. JPL

In [ ]:
df_earth_mse

In [ ]:
# Run comparison on earth
compare_df_vec(df_mse=df_earth_mse, df_jpl=df_earth_jpl, name='earth')

### Check Position of Asteroids vs. JPL

In [ ]:
df_ast_mse

In [ ]:
# Filter MSE asteroids down to just the first 16 to match JPL data
mask = df_ast_mse.asteroid_num <= 16
df_ast_mse_16 = df_ast_mse[mask]

In [ ]:
# Run comparison on asteroids
compare_df_vec(df_mse=df_ast_mse_16, df_jpl=df_ast_jpl, name='asteroids')

### Review Solar DataFrame

In [ ]:
df_sun_mse

**Conclusion**<br>
Loading daily integration with load_ast_data works.<br>
Integration agrees with JPL to tolerance of **7.9E-7 AU**.

### Splined Asteroid DataFrame

In [ ]:
# Load the JPL data run at 3 hour intervals
df_earth_jpl_3h = load_pos_jpl(body_name='earth', dir_name=dir_name_hourly)
df_ast_jpl_3h = load_ast_pos_jpl(ast_num0=1, ast_num1=16, dir_name=dir_name_hourly)

# Load observation from palomar at 3h intervals
df_dir_jpl_3h = load_ast_obs_jpl(ast_num0=1, ast_num1=16, observer_name='palomar', dir_name=dir_name_hourly)

In [ ]:
# Inputs for spline_ast_vec()
n0 = 1
n1 = 17
mjd = df_earth_jpl_3h.mjd.values

In [ ]:
# Spline asteroid data on the same schedule as JPL
df_ast_out, df_earth_out, df_sun_out = spline_ast_vec(n0=n0, n1=n1, mjd=mjd)

In [ ]:
df_ast_out

In [ ]:
# Run comparison on asteroids
compare_df_vec(df_mse=df_ast_out, df_jpl=df_ast_jpl_3h, name='asteroids')

In [ ]:
df_earth_out

In [ ]:
# Run comparison on earth
compare_df_vec(df_mse=df_earth_out, df_jpl=df_earth_jpl_3h, name='earth')

**Conclusion**<br>
Cubic splining of daily integration with spline_ast_vec works.<br>
Integration agrees with JPL to tolerance of **3.9E-6 AU**.<br>
The spline has introduced slightly more error, but it is still very small.

### Build Splined Observation & Compare vs. JPL

In [ ]:
df_dir_jpl_3h

In [ ]:
# Calulate observations at palomar from splined vectors
df_dir_mse = calc_ast_dir(df_ast=df_ast_out, df_earth=df_earth_out, site_name='palomar')

In [ ]:
df_dir_mse

In [ ]:
compare_df_dir(df_dir_mse, df_dir_jpl_3h, name='asteroid')

**Conclusion**<br>
df_obs_mse() accurately does an end to end calculation of the RA / DEC of an asteroid.<br>
It is very flexible.  It interpolated the position of the asteroid and the position of the earth,
so it can accept a flexible range of dates to match observations.<br>
It is accurate vs. JPL to a tolerance of **0.97 arc seconds**!

### Test All in One spline_ast_vec_obs()

In [ ]:
mjd = df_earth_jpl_3h.mjd.values
df_ast2_mse, df_earth2_mse, df_dir2_mse = spline_ast_vec_dir(n0=1, n1=17, mjd=mjd, site_name='palomar')

In [ ]:
compare_df_dir(df_dir2_mse, df_dir_jpl_3h, name='asteroid')

## calc_ast_data: Build Asteroid DataFrame on the Fly

In [ ]:
n0=1
n1=65
mjd0 = 58270.0
mjd1 = 58906.0
progbar=True

In [ ]:
# Load data in this date range
df_ast_load, df_earth_load, df_sun_load = load_ast_data(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1, progbar=progbar)

In [ ]:
df_ast_load

In [ ]:
# Orbital elements for first 64 asteroids (dict of numpy arrays)
# n0 = 733
n0 = 1
n1 = n0 + 64
element_id = np.arange(n0, n1, dtype=np.int32)

# Build the elements batch
elts = orbital_element_batch(element_id)

In [ ]:
# Calculate vectors from these elements on the fly with calc_ast_data
df_ast_calc, df_earth_calc, df_sun_calc = calc_ast_data(elts=elts, mjd0=mjd0, mjd1=mjd1, element_id=element_id)

In [ ]:
# Check agreement
cols_q = ['qx', 'qy', 'qz']
cols_v = ['vx', 'vy', 'vz']

dq_ast = df_ast_calc[cols_q] - df_ast_load[cols_q]
dv_ast = df_ast_calc[cols_v] - df_ast_load[cols_v]

dq_earth = df_earth_calc[cols_q] - df_earth_load[cols_q]
dv_earth = df_earth_calc[cols_v] - df_earth_load[cols_v]

dq_sun = df_sun_calc[cols_q] - df_sun_load[cols_q]
dv_sun = df_sun_calc[cols_v] - df_sun_load[cols_v]

# Report
# print(f'mean position difference = {np.mean(np.linalg.norm(dq, axis=1)):6.2e}')
print('Mean position difference in AU:')
print(f'asteroids: {np.mean(np.linalg.norm(dq_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dq_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dq_sun, axis=1)):6.2e}')

print('\nMean velocity difference in AU/day:')
print(f'asteroids: {np.mean(np.linalg.norm(dv_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dv_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dv_sun, axis=1)):6.2e}')

## Splined Direction at ZTF Dates

In [ ]:
ztf, mjd_unq = load_ztf_det_all()

In [ ]:
# mjd for the spline output
mjd = mjd_unq

In [ ]:
# Spline using loaded data
df_ast_load_out, df_earth_load_out, df_sun_load_out = spline_ast_vec_df(df_ast=df_ast_load, df_earth=df_earth_load, df_sun=df_sun_load, mjd=mjd, include_elts=False)

In [ ]:
# Spline using calculated data
df_ast_calc_out, df_earth_calc_out, df_sun_calc_out = spline_ast_vec_df(df_ast=df_ast_calc, df_earth=df_earth_calc, df_sun=df_sun_calc, mjd=mjd, include_elts=False)

In [ ]:
dq_ast = df_ast_calc_out[cols_q] - df_ast_load_out[cols_q]
dv_ast = df_ast_calc_out[cols_v] - df_ast_load_out[cols_v]

dq_earth = df_earth_calc_out[cols_q] - df_earth_load_out[cols_q]
dv_earth = df_earth_calc_out[cols_v] - df_earth_load_out[cols_v]

dq_sun = df_sun_calc_out[cols_q] - df_sun_load_out[cols_q]
dv_sun = df_sun_calc_out[cols_v] - df_sun_load_out[cols_v]

# Report
# print(f'mean position difference = {np.mean(np.linalg.norm(dq, axis=1)):6.2e}')
print('Mean position difference in AU:')
print(f'asteroids: {np.mean(np.linalg.norm(dq_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dq_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dq_sun, axis=1)):6.2e}')

print('\nMean velocity difference in AU/day:')
print(f'asteroids: {np.mean(np.linalg.norm(dv_ast, axis=1)):6.2e}')
print(f'earth    : {np.mean(np.linalg.norm(dv_earth, axis=1)):6.2e}')
print(f'sun      : {np.mean(np.linalg.norm(dv_sun, axis=1)):6.2e}')

In [ ]:
df_ast_calc_out